In [11]:
import os
import pandas as pd
from pathlib import Path
from fredapi import Fred
import sys
sys.path.append('../')
from src.data import fred_data

We set FED key

In [2]:
fred = Fred(os.getenv("API_FED"))
fred

It is time to create a dict of the vars and their codes to be downloaded by the FED API

In [3]:
codes_fed = {
    "crude_oil_wti" : "DCOILWTICO",
    "crude_oil_brent" : "DCOILBRENTEU",
    "gas_henry_hub" : "DHHNGSP",
    "nominal_broad_usdollar" : "DTWEXBGS",
    "yuan_dollar_spot_tc" : "DEXCHUS",
    "usdollar_euro" : "DEXUSEU",
    "treasury3month" : "DTB3",
    "treasury1year" : "DTB1YR",
    "treasury4week" : "DTB4WK",
    "treasury6month" : "DTB6"
}

We do the call

In [4]:
data_fred = {}
for k, v in codes_fed.items():
    data_fred[k] = fred.get_series(v)

In [5]:
data_fred["crude_oil_wti"]

1986-01-02    25.56
1986-01-03    26.00
1986-01-06    26.53
1986-01-07    25.85
1986-01-08    25.87
              ...  
2022-08-09    93.18
2022-08-10    94.68
2022-08-11    97.02
2022-08-12    94.86
2022-08-15    92.24
Length: 9553, dtype: float64

For creating the correct data frame we generate a fun that the logic is the same in the next code:

In [6]:
temp = data_fred["crude_oil_wti"].to_frame().reset_index()
temp = (temp.rename(columns={temp.columns[0] : "date",
                    temp.columns[1] : "value"})
            .assign(variable = "crude_oil_wti")
        )
temp

,date,value,variable
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
9548,2022-08-09,93.18,crude_oil_wti
9549,2022-08-10,94.68,crude_oil_wti
9550,2022-08-11,97.02,crude_oil_wti
9551,2022-08-12,94.86,crude_oil_wti


Here, we apply the fun

In [7]:
data_fred_clean = {}
for k, v in codes_fed.items():
    data_fred_clean[k] = fred_data.cleaning_fred(ds=data_fred[k], var_name=k)

In [8]:
data_fred_clean['gas_henry_hub']

,date,value,variable
0,1997-01-07,3.82,gas_henry_hub
1,1997-01-08,3.80,gas_henry_hub
2,1997-01-09,3.61,gas_henry_hub
3,1997-01-10,3.92,gas_henry_hub
4,1997-01-13,4.00,gas_henry_hub
...,...,...,...
6681,2022-08-10,7.86,gas_henry_hub
6682,2022-08-11,8.53,gas_henry_hub
6683,2022-08-12,8.73,gas_henry_hub
6684,2022-08-15,8.62,gas_henry_hub


Concat the dataframes

In [9]:
raw_fred = pd.concat(data_fred_clean, axis=0, ignore_index=True)
raw_fred

,date,value,variable
0,1986-01-02,25.56,crude_oil_wti
1,1986-01-03,26.00,crude_oil_wti
2,1986-01-06,26.53,crude_oil_wti
3,1986-01-07,25.85,crude_oil_wti
4,1986-01-08,25.87,crude_oil_wti
...,...,...,...
103275,2022-08-15,3.02,treasury6month
103276,2022-08-16,3.01,treasury6month
103277,2022-08-17,3.03,treasury6month
103278,2022-08-18,3.01,treasury6month


Saving the dataframe

In [18]:
fred_path = Path.cwd().parent.joinpath("data", "raw_fred.csv")
print(f"output path is:{fred_path}")
raw_fred.to_csv(fred_path, index=False)

output path is:c:\Documentos\proyectos\xm4ecast\data\raw_fred.csv
